# News Headlines Text Generation

### Load Dataset

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import os, string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
articles_dir = 'Articles'
all_headlines = []
for filename in os.listdir(articles_dir):
    df = pd.read_csv(os.path.join(articles_dir, filename))
    all_headlines.extend(list(df['headline'].values))

all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

### Preprocess

In [ ]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode('utf8').decode('ascii', 'ignore')
    return txt 

corpus = [clean_text(x) for x in all_headlines]

In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = to_categorical(labels, num_classes=total_words)

### Model Training + Evaluation

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
history = model.fit(xs, ys, epochs=100)
print(model)

### Generation

In [ ]:
def generate_text(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

In [ ]:
print(generate_text("united states", 5))
print(generate_text("preident trump", 4))
print(generate_text("donald trump", 4))
print(generate_text("india and china", 4))
print(generate_text("new york", 4))
print(generate_text("science and technology", 5))

In [ ]:
'''
Inspiration
1. https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms
2. https://www.coursera.org/learn/natural-language-processing-tensorflow?specialization=tensorflow-in-practice
'''